In [42]:
import pytesseract
import re
import openpyxl as opxl
from PIL import Image, ImageDraw
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re as regex
from collections import defaultdict
import cv2

In [61]:
import cv2

refPt = []
coords = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping, coords, image

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        if len(refPt) == 2:
            x_start, y_start = refPt[0]
            x_end, y_end = refPt[1]
            width = x_end - x_start
            height = y_end - y_start
            bbox = (x_start, y_start, width, height)
            coords.append(bbox)

            cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
            cv2.imshow("image", image)

        refPt = []

image = cv2.imread('sample2.jpg')
clone = image.copy()

cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("r"):
        image = clone.copy()
        coords.clear()
    elif key == ord("q"):
        break

cv2.destroyAllWindows()

for i, box in enumerate(coords, 1):
    print(f"{i}: {box}")

1: (321, 40, 178, 82)


In [62]:
def string_conversion():
    pass


image = Image.open('sample2.jpg')
placeholders = ['', ' ']
for box_coords in coords:
    new_image = image.crop((box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[3] + box_coords[1]))
    ocr_data = pytesseract.image_to_data(new_image, output_type=pytesseract.Output.DICT)
    print(ocr_data)
    ocr_data['text'] = [o for o in ocr_data['text'] if o not in placeholders]
    string = ' '.join(ocr_data['text'])
    print(ocr_data['text'])
    print(string)

{'level': [1, 2, 3, 4, 5, 5, 4, 5, 5, 3, 4, 5, 5, 3, 4, 5, 5, 5, 3, 4, 5, 5, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4], 'line_num': [0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 'word_num': [0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 4], 'left': [0, 7, 7, 7, 7, 55, 7, 7, 55, 7, 7, 7, 56, 7, 7, 7, 55, 82, 7, 7, 7, 32, 69, 103], 'top': [0, 7, 7, 7, 7, 7, 23, 23, 23, 38, 38, 38, 39, 54, 54, 54, 54, 42, 70, 70, 70, 71, 58, 70], 'width': [178, 156, 64, 59, 23, 11, 64, 32, 16, 86, 86, 25, 37, 107, 107, 41, 28, 32, 156, 156, 20, 31, 8, 60], 'height': [82, 70, 24, 7, 7, 7, 8, 8, 7, 8, 8, 8, 6, 9, 9, 7, 7, 28, 7, 7, 7, 6, 24, 7], 'conf': [-1, -1, -1, -1, 59, 55, -1, 0, 22, -1, -1, 37, 0, -1, -1, 35, 44, 33, -1, -1, 37, 30, 